# Introduction

THis notebook contains he final notebook submission to the Kaggle competion predcit Calories challenge. I will start by first havoing code to the best possible solution based on the public leaderboard) and then explore other options. EDA and other stuff have been removed from this notebook for readability.

In [1]:
# imports

import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('PredictCalorie/train.csv')
test = pd.read_csv('PredictCalorie/test.csv')
sample = pd.read_csv('PredictCalorie/sample_submission.csv')

# Use of CatBoost

In [16]:
# Define columns to exclude from features
exclude_cols = ['id', 'Calories']
target_col = 'Calories'

# Split features and target from train
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Prepare test set (same feature columns)
X_test = test.drop(columns=['id'])

# Identify categorical features (assumes object types are categorical)
categorical_features = ['Sex']

# Optional: split validation set from train (for early stopping)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define Pools
train_pool = Pool(X_tr, y_tr, cat_features=categorical_features)
val_pool = Pool(X_val, y_val, cat_features=categorical_features)
test_pool = Pool(X_test, cat_features=categorical_features)

# Train the model
model = CatBoostRegressor(
    iterations=2000,
    learning_rate=0.03,
    depth=10,
    random_strength=5,
    l2_leaf_reg=7,
    bootstrap_type='Bernoulli',
    grow_policy='SymmetricTree',
    eval_metric='RMSE',
    random_seed=42,
    early_stopping_rounds=50,
    verbose=100
)

model.fit(train_pool, eval_set=val_pool)

# Predict on test set
test_preds = model.predict(test_pool)

# Add predictions to test dataframe
test['Calories'] = test_preds

# Optional: preview results
print(test[['id','Calories']].head())
test[['id','Calories']].to_csv('catboost.csv',index=False)

0:	learn: 60.6811318	test: 60.5082800	best: 60.5082800 (0)	total: 36.1ms	remaining: 1m 12s
100:	learn: 5.7573990	test: 5.7426415	best: 5.7426415 (100)	total: 1.49s	remaining: 28.1s
200:	learn: 3.8117644	test: 3.8401566	best: 3.8401566 (200)	total: 3.02s	remaining: 27s
300:	learn: 3.6295186	test: 3.6791427	best: 3.6791427 (300)	total: 4.77s	remaining: 26.9s
400:	learn: 3.5529231	test: 3.6217363	best: 3.6217363 (400)	total: 6.49s	remaining: 25.9s
500:	learn: 3.5078001	test: 3.5929045	best: 3.5929045 (500)	total: 8.19s	remaining: 24.5s
600:	learn: 3.4760466	test: 3.5780044	best: 3.5780044 (600)	total: 9.86s	remaining: 23s
700:	learn: 3.4526859	test: 3.5672550	best: 3.5672550 (700)	total: 11.6s	remaining: 21.5s
800:	learn: 3.4337295	test: 3.5609999	best: 3.5609999 (800)	total: 13.3s	remaining: 19.9s
900:	learn: 3.4170465	test: 3.5564010	best: 3.5564010 (900)	total: 15s	remaining: 18.3s
1000:	learn: 3.4020588	test: 3.5525224	best: 3.5525209 (999)	total: 16.8s	remaining: 16.8s
1100:	learn: 3

In [74]:
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np

# Bin 'Duration' into quantile-based categories (e.g., 5 groups)
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Add Duration binning for stratification
duration_bins = pd.qcut(train['Duration'], q=10, labels=False)  # or use pd.cut for fixed bins

# Initialize StratifiedKFold using duration bins
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

test_preds = np.zeros(len(test))


# Proceed with the same logic — just replace KFold with StratifiedKFold
for fold, (idx_train, idx_valid) in enumerate(cv.split(X_train, duration_bins)):
    print(f'\nFold {fold + 1}')
    
    X_tr, X_val = X_train.iloc[idx_train], X_train.iloc[idx_valid]
    y_tr, y_val = np.log1p(y_train.iloc[idx_train]), np.log1p(y_train.iloc[idx_valid])
    
    # CatBoost Pools
    train_pool = Pool(X_tr, y_tr, cat_features=categorical_features)
    val_pool = Pool(X_val, y_val, cat_features=categorical_features)
    test_pool = Pool(X_test, cat_features=categorical_features)
    
    # Train and predict as before...
    model = CatBoostRegressor(
        iterations=2000,
        learning_rate=0.03,
        depth=10,
        random_strength=5,
        l2_leaf_reg=7,
        bootstrap_type='Bernoulli',
        grow_policy='SymmetricTree',
        eval_metric='RMSE',
        random_seed=42,
        early_stopping_rounds=50,
        verbose=100
    )
    
    model.fit(train_pool, eval_set=val_pool)
    test_preds += np.expm1(model.predict(test_pool))

test_preds /= cv.get_n_splits()
test['Calories'] = test_preds
test[['id','Calories']].to_csv('catboost_5_fold_str.csv',index=False)


Fold 1
0:	learn: 0.9366100	test: 0.9354519	best: 0.9354519 (0)	total: 33.6ms	remaining: 1m 7s
100:	learn: 0.0924376	test: 0.0926483	best: 0.0926483 (100)	total: 1.57s	remaining: 29.5s
200:	learn: 0.0637060	test: 0.0646417	best: 0.0646417 (200)	total: 3.27s	remaining: 29.2s
300:	learn: 0.0606240	test: 0.0618747	best: 0.0618747 (300)	total: 5.35s	remaining: 30.2s
400:	learn: 0.0595081	test: 0.0610308	best: 0.0610305 (399)	total: 7.21s	remaining: 28.7s
500:	learn: 0.0588160	test: 0.0605886	best: 0.0605886 (500)	total: 9.31s	remaining: 27.8s
600:	learn: 0.0583254	test: 0.0603792	best: 0.0603782 (599)	total: 11s	remaining: 25.7s
700:	learn: 0.0579711	test: 0.0602938	best: 0.0602938 (700)	total: 12.8s	remaining: 23.7s
800:	learn: 0.0576959	test: 0.0602223	best: 0.0602223 (800)	total: 14.7s	remaining: 22s
900:	learn: 0.0574552	test: 0.0601871	best: 0.0601871 (900)	total: 16.4s	remaining: 20s
1000:	learn: 0.0572292	test: 0.0601481	best: 0.0601469 (991)	total: 18.4s	remaining: 18.4s
1100:	lear

# Add more features

In [43]:
train = pd.read_csv('PredictCalorie/train.csv')
test = pd.read_csv('PredictCalorie/test.csv')
sample = pd.read_csv('PredictCalorie/sample_submission.csv')
train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [44]:
train['bmi']=train['Weight']/(train['Height']*train['Height'])*100*100
test['bmi']=test['Weight']/(test['Height']*test['Height'])*100*100

train['dur_body_heart']=train['Duration']*train['Heart_Rate']*train['Body_Temp']
test['dur_body_heart']=test['Duration']*test['Heart_Rate']*test['Body_Temp']



In [48]:
def calculate_bmr(row):
    if row['Sex'].lower() == 'male':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] + 5
    elif row['Sex'].lower() == 'female':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] - 161
    else:
        return np.nan  # handle unknown or missing Sex

train['BMR'] = train.apply(calculate_bmr, axis=1)
test['BMR'] = test.apply(calculate_bmr, axis=1)

In [49]:
train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,bmi,dur_body_heart,BMR
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0,22.955684,107666.0,1826.25
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0,22.582709,26996.0,1137.75
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0,24.690405,23402.4,1230.25
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0,24.414062,106837.5,2005.00
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0,22.136740,103530.0,1296.50


# XGB Regressor

In [79]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import TargetEncoder
import xgboost as xgb
import numpy as np
import pandas as pd

categorical_cols = ['Sex']

# Discretize 'duration' into 5 bins for stratification
duration_bins = pd.qcut(X['Duration'], q=5, labels=False, duplicates='drop')

# Use StratifiedKFold for stratified splitting based on duration bins
cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
y_pred = np.zeros(len(test))

# Log-transform the target
y_log = np.log1p(y)  # log(1 + y) for RMSLE-friendly training

for idx_train, idx_valid in cv.split(X, duration_bins):
    X_train, y_train = X.iloc[idx_train].copy(), y_log.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid].copy(), y_log.iloc[idx_valid]
    X_test = test[X.columns].copy()

    # Target Encoding for categorical column
    encoder = TargetEncoder(random_state=42)
    X_train['Sex'] = encoder.fit_transform(X_train[['Sex']], y_train)[:, 0]
    X_valid['Sex'] = encoder.transform(X_valid[['Sex']])[:, 0]
    X_test['Sex'] = encoder.transform(X_test[['Sex']])[:, 0]

    model = xgb.XGBRegressor(
        n_estimators=2000,
        max_depth=10,
        learning_rate=0.03,
        subsample=0.7,
        colsample_bytree=0.7,
        objective='reg:squarederror',  # L2 loss
        eval_metric='rmse',
        early_stopping_rounds=100,
        random_state=42,
        verbosity=1,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=100
    )

    y_pred += np.expm1(model.predict(X_test))

# Average predictions over folds
y_pred /= cv.get_n_splits()

# Create submission file
pred_xgb = y_pred
sample_submission = pd.read_csv('sample_submission.csv')
submission_xgb = pd.DataFrame({'id': sample_submission.id, 'Calories': pred_xgb})
submission_xgb.to_csv('submission_xgb.csv', index=False)

[0]	validation_0-rmse:0.93585
[100]	validation_0-rmse:0.09281
[200]	validation_0-rmse:0.06295
[300]	validation_0-rmse:0.06119
[400]	validation_0-rmse:0.06072
[500]	validation_0-rmse:0.06057
[600]	validation_0-rmse:0.06059
[615]	validation_0-rmse:0.06061
[0]	validation_0-rmse:0.93293
[100]	validation_0-rmse:0.09143
[200]	validation_0-rmse:0.06255
[300]	validation_0-rmse:0.06088
[400]	validation_0-rmse:0.06040
[500]	validation_0-rmse:0.06028
[600]	validation_0-rmse:0.06030
[653]	validation_0-rmse:0.06032
[0]	validation_0-rmse:0.93667
[100]	validation_0-rmse:0.09201
[200]	validation_0-rmse:0.06195
[300]	validation_0-rmse:0.06014
[400]	validation_0-rmse:0.05969
[500]	validation_0-rmse:0.05959
[600]	validation_0-rmse:0.05961
[629]	validation_0-rmse:0.05962
[0]	validation_0-rmse:0.93631
[100]	validation_0-rmse:0.09176
[200]	validation_0-rmse:0.06168
[300]	validation_0-rmse:0.05986
[400]	validation_0-rmse:0.05933
[500]	validation_0-rmse:0.05916
[600]	validation_0-rmse:0.05915
[652]	validation

# ENsemble the two best cases (In log space)

In [87]:
data1=pd.read_csv('submission_xgb (1).csv')
data2=pd.read_csv('catboost_5_fold_str (1).csv')

data = data1.merge(data2, on='id', suffixes=('_xgb', '_cat'))

In [88]:
data.head()

,id,Calories_xgb,Calories_cat
0,750000,27.488697,27.206506
1,750001,107.910565,107.878140
2,750002,87.331006,87.098264
3,750003,126.202942,125.542912
4,750004,76.272044,76.180985


In [89]:
import pandas as pd
import numpy as np



# 1. Simple average of the two predictions
data['Calories_avg'] = (data['Calories_xgb'] + data['Calories_cat']) / 2

# 2. Log-scale average (RMSLE-style averaging)
data['Calories_log_avg'] = np.expm1(
    (np.log1p(data['Calories_xgb']) + np.log1p(data['Calories_cat'])) / 2
)

print(data[['id', 'Calories_xgb', 'Calories_cat', 'Calories_avg', 'Calories_log_avg']].head())

       id  Calories_xgb  Calories_cat  Calories_avg  Calories_log_avg
0  750000     27.488697     27.206506     27.347601         27.347250
1  750001    107.910565    107.878140    107.894353        107.894351
2  750002     87.331006     87.098264     87.214635         87.214558
3  750003    126.202942    125.542912    125.872927        125.872498
4  750004     76.272044     76.180985     76.226515         76.226501


In [90]:
# Save simple average with 'Calories' as the second column
data[['id', 'Calories_avg']].rename(columns={'Calories_avg': 'Calories'}).to_csv('submission_reg_avg.csv', index=False)

# Save log-scale average with 'Calories' as the second column
data[['id', 'Calories_log_avg']].rename(columns={'Calories_log_avg': 'Calories'}).to_csv('submission_log_avg.csv', index=False)

# Start fresh with new features

In [75]:
train = pd.read_csv('PredictCalorie/train.csv')
test = pd.read_csv('PredictCalorie/test.csv')
sample = pd.read_csv('PredictCalorie/sample_submission.csv')



In [80]:
train['bmi']=train['Weight']/(train['Height']*train['Height'])*100*100
test['bmi']=test['Weight']/(test['Height']*test['Height'])*100*100

train['dur_body_heart']=train['Duration']*train['Heart_Rate']*train['Body_Temp']
test['dur_body_heart']=test['Duration']*test['Heart_Rate']*test['Body_Temp']

def calculate_bmr(row):
    if row['Sex'].lower() == 'male':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] + 5
    elif row['Sex'].lower() == 'female':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] - 161
    else:
        return np.nan  # handle unknown or missing Sex

train['BMR'] = train.apply(calculate_bmr, axis=1)
test['BMR'] = test.apply(calculate_bmr, axis=1)

# Squares of features
train['dur_2']=train['Duration']*train['Duration']
test['dur_2']=test['Duration']*test['Duration']

train['hr_2']=train['Heart_Rate']*train['Heart_Rate']
test['hr_2']=test['Heart_Rate']*test['Heart_Rate']

train['height_2']=train['Height']*train['Height']
test['height_2']=test['Height']*test['Height']

train['weight_2']=train['Weight']*train['Weight']
test['weight_2']=test['Weight']*test['Weight']

# Square root of Duration
train['dur_sqrt'] = train['Duration'] ** 0.5
test['dur_sqrt'] = test['Duration'] ** 0.5

train['hr_sqrt'] = train['Heart_Rate'] ** 0.5
test['hr_sqrt'] = test['Heart_Rate'] ** 0.5

train['weight_sqrt'] = train['Weight'] ** 0.5
test['weight_sqrt'] = test['Weight'] ** 0.5

train['height_sqrt'] = train['Height'] ** 0.5
test['height_sqrt'] = test['Height'] ** 0.5

train['Intensity'] = train['Heart_Rate'] / train['Duration']
test['Intensity'] = test['Heart_Rate'] / test['Duration']

# Create binary indicators
train['is_male'] = (train['Sex'].str.lower() == 'male').astype(int)
train['is_female'] = (train['Sex'].str.lower() == 'female').astype(int)

test['is_male'] = (test['Sex'].str.lower() == 'male').astype(int)
test['is_female'] = (test['Sex'].str.lower() == 'female').astype(int)

# Numerical columns to interact with sex
num_cols = ['Age', 'Duration', 'Heart_Rate', 'Weight', 'Height', 'Body_Temp','bmi','Intensity','dur_body_heart']

# Create interaction features
for col in num_cols:
    train[f'{col}_Male'] = train[col] * train['is_male']
    train[f'{col}_Female'] = train[col] * train['is_female']

    test[f'{col}_Male'] = test[col] * test['is_male']
    test[f'{col}_Female'] = test[col] * test['is_female']


for col in ['Weight', 'Height', 'Duration', 'Heart_Rate', 'Body_Temp']:
    train[f'{col}_log'] = np.log1p(train[col])
    test[f'{col}_log'] = np.log1p(test[col])

train['Weight_Height_ratio'] = train['Weight'] / train['Height']
test['Weight_Height_ratio'] = test['Weight'] / test['Height']

train['HeartRate_BodyTemp_ratio'] = train['Heart_Rate'] / train['Body_Temp']
test['HeartRate_BodyTemp_ratio'] = test['Heart_Rate'] / test['Body_Temp']


train['effort_per_kg'] = train['Duration'] * train['Heart_Rate'] / train['Weight']
test['effort_per_kg'] = test['Duration'] * test['Heart_Rate'] / test['Weight']


train['Age_HeartRate'] = train['Age'] * train['Heart_Rate']
test['Age_HeartRate'] = test['Age'] * test['Heart_Rate']

train['Temp_Height'] = train['Body_Temp'] * train['Height']
test['Temp_Height'] = test['Body_Temp'] * test['Height']

from itertools import combinations

poly_cols = ['Age', 'Duration', 'Heart_Rate', 'Weight', 'Height']
for col1, col2 in combinations(poly_cols, 2):
    train[f'{col1}_x_{col2}'] = train[col1] * train[col2]
    test[f'{col1}_x_{col2}'] = test[col1] * test[col2]
'''
train['Duration_2'] = train['Duration']
test['Duration_2'] = test['Duration']
'''
train['Age_2'] = train['Age']
test['Age_2'] = test['Age']


In [24]:
train.columns

Index(['id', 'Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate',
       'Body_Temp', 'Calories'],
      dtype='object')

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import TargetEncoder
import xgboost as xgb
import numpy as np
import pandas as pd


drop_cols = ['id', 'Calories']
X = train.drop(columns=drop_cols)
y = train['Calories']
cat_features = ['Sex']

categorical_cols = ['Sex']

# Discretize 'duration' into 5 bins for stratification
duration_bins = pd.qcut(X['Duration'], q=10, labels=False, duplicates='drop')

# Use StratifiedKFold for stratified splitting based on duration bins
cv = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
y_pred = np.zeros(len(test))

# Log-transform the target
y_log = np.log1p(y)  # log(1 + y) for RMSLE-friendly training

for idx_train, idx_valid in cv.split(X, duration_bins):
    X_train, y_train = X.iloc[idx_train].copy(), y_log.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid].copy(), y_log.iloc[idx_valid]
    X_test = test[X.columns].copy()

    # Target Encoding for categorical column
    encoder = TargetEncoder(random_state=42)
    X_train['Sex'] = encoder.fit_transform(X_train[['Sex']], y_train)[:, 0]
    X_valid['Sex'] = encoder.transform(X_valid[['Sex']])[:, 0]
    X_test['Sex'] = encoder.transform(X_test[['Sex']])[:, 0]

    model = xgb.XGBRegressor(
        n_estimators=2000,
        max_depth=10,
        learning_rate=0.03,
        subsample=0.7,
        colsample_bytree=0.7,
        objective='reg:squarederror',  # L2 loss
        eval_metric='rmse',
        early_stopping_rounds=100,
        random_state=42,
        verbosity=1,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=100
    )

    y_pred += np.expm1(model.predict(X_test))

# Average predictions over folds
y_pred /= cv.get_n_splits()

# Create submission file
pred_xgb = y_pred
sample_submission = pd.read_csv('sample_submission.csv')
submission_xgb = pd.DataFrame({'id': sample_submission.id, 'Calories': pred_xgb})
submission_xgb.to_csv('submission_xgb_new.csv', index=False)

In [69]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import TargetEncoder
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import numpy as np
import pandas as pd

# Setup
drop_cols = ['id', 'Calories']
X = train.drop(columns=drop_cols)
y = train['Calories']
cat_features = ['Sex','Duration','Age']

# Stratification based on binned Duration
duration_bins = pd.qcut(X['Duration'], q=10, labels=False, duplicates='drop')

# Stratified K-Fold
cv = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

# Log-transform the target
y_log = np.log1p(y)

# Initialize predictions
y_pred = np.zeros(len(test))
oof_preds = np.zeros(len(train))  # To store OOF predictions

for idx_fold, (idx_train, idx_valid) in enumerate(cv.split(X, duration_bins)):
    print(f"\nFold {idx_fold + 1}")
    
    X_train, y_train_fold = X.iloc[idx_train].copy(), y_log.iloc[idx_train]
    X_valid, y_valid_fold = X.iloc[idx_valid].copy(), y_log.iloc[idx_valid]
    X_test = test[X.columns].copy()

    # Target encoding
    encoder = TargetEncoder(random_state=42)
    X_train['Sex'] = encoder.fit_transform(X_train[['Sex']], y_train_fold)[:, 0]
    X_valid['Sex'] = encoder.transform(X_valid[['Sex']])[:, 0]
    X_test['Sex'] = encoder.transform(X_test[['Sex']])[:, 0]

    # Model
    model = xgb.XGBRegressor(
        n_estimators=2000,
        max_depth=10,
        learning_rate=0.03,
        subsample=0.7,
        colsample_bytree=0.7,
        objective='reg:squarederror',
        eval_metric='rmse',
        early_stopping_rounds=100,
        random_state=42,
        verbosity=1,
    )

    model.fit(
        X_train, y_train_fold,
        eval_set=[(X_valid, y_valid_fold)],
        verbose=100
    )

    # Predict on test and validation sets
    y_pred += np.expm1(model.predict(X_test))
    oof_preds[idx_valid] = np.expm1(model.predict(X_valid))

# Final test prediction (average)
y_pred /= cv.get_n_splits()

# OOF RMSLE
rmsle = np.sqrt(mean_squared_error(np.log1p(train['Calories']), np.log1p(oof_preds)))
print(f"\nOOF RMSLE: {rmsle:.5f}")

# Submission
sample_submission = pd.read_csv('sample_submission.csv')
submission_xgb = pd.DataFrame({'id': sample_submission.id, 'Calories': y_pred})
submission_xgb.to_csv('submission_xgb_new.csv', index=False)


Fold 1
[0]	validation_0-rmse:0.93366
[100]	validation_0-rmse:0.07485
[200]	validation_0-rmse:0.05952
[300]	validation_0-rmse:0.05947
[355]	validation_0-rmse:0.05951

Fold 2
[0]	validation_0-rmse:0.93360
[100]	validation_0-rmse:0.07593
[200]	validation_0-rmse:0.06049
[300]	validation_0-rmse:0.06042
[370]	validation_0-rmse:0.06044

Fold 3
[0]	validation_0-rmse:0.93571
[100]	validation_0-rmse:0.07493
[200]	validation_0-rmse:0.05893
[300]	validation_0-rmse:0.05888
[337]	validation_0-rmse:0.05890

Fold 4
[0]	validation_0-rmse:0.93432
[100]	validation_0-rmse:0.07461
[200]	validation_0-rmse:0.05908
[300]	validation_0-rmse:0.05909
[339]	validation_0-rmse:0.05910

Fold 5
[0]	validation_0-rmse:0.93583
[100]	validation_0-rmse:0.07569
[200]	validation_0-rmse:0.06020
[300]	validation_0-rmse:0.06018
[337]	validation_0-rmse:0.06019

Fold 6
[0]	validation_0-rmse:0.93168
[100]	validation_0-rmse:0.07557
[200]	validation_0-rmse:0.05996
[300]	validation_0-rmse:0.05990
[360]	validation_0-rmse:0.05996

Fol

In [7]:
OOF RMSLE: 0.05991 => with ratio features
OOF RMSLE: 0.05993 => with ratio features + efficiency
OOF RMSLE: 0.05988 => with ratio features + efficiency + interation
OOF RMSLE: 0.05985 => with ratio features + efficiency + interation + pairwise polynomial
OOF RMSLE: 0.05994 => with ratio features +            + interation + pairwise polynomial


OOF RMSLE: 0.05983 => with ratio features +  effciency + interation + pairwise polynomial with Duration as a cxategory
OOF RMSLE: 0.05983 => with ratio features +  effciency + interation + pairwise polynomial with Duration and Age as a cxategory



750000

In [84]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import TargetEncoder
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import numpy as np
import pandas as pd

# Setup
drop_cols = ['id', 'Calories']
X = train.drop(columns=drop_cols)
y = train['Calories']
cat_features = ['Sex','Duration','Age']
categorical_cols = ['Sex','Duration']#,'Age']
# Stratification based on binned Duration
duration_bins = pd.qcut(X['Duration'], q=10, labels=False, duplicates='drop')

# Stratified K-Fold
cv = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

# Log-transform the target
y_log = np.log1p(y)

# Initialize predictions
y_pred = np.zeros(len(test))
oof_preds = np.zeros(len(train))  # To store OOF predictions

for idx_fold, (idx_train, idx_valid) in enumerate(cv.split(X, duration_bins)):
    print(f"\nFold {idx_fold + 1}")
    
    X_train, y_train_fold = X.iloc[idx_train].copy(), y_log.iloc[idx_train]
    X_valid, y_valid_fold = X.iloc[idx_valid].copy(), y_log.iloc[idx_valid]
    X_test = test[X.columns].copy()

    encoder = TargetEncoder(random_state=42)
    X_train[categorical_cols] = encoder.fit_transform(X_train[categorical_cols], y_train_fold)
    X_valid[categorical_cols] = encoder.transform(X_valid[categorical_cols])
    X_test[categorical_cols] = encoder.transform(X_test[categorical_cols])

    # Model
    model = xgb.XGBRegressor(
        n_estimators=2000,
        max_depth=10,
        learning_rate=0.03,
        subsample=0.7,
        colsample_bytree=0.7,
        objective='reg:squarederror',
        eval_metric='rmse',
        early_stopping_rounds=100,
        random_state=42,
        verbosity=1,
    )

    model.fit(
        X_train, y_train_fold,
        eval_set=[(X_valid, y_valid_fold)],
        verbose=100
    )

    # Predict on test and validation sets
    y_pred += model.predict(X_test)
    oof_preds[idx_valid] = np.expm1(model.predict(X_valid))

# Final test prediction (average)
y_pred = np.expm1(y_pred / cv.get_n_splits()) 

# OOF RMSLE
rmsle = np.sqrt(mean_squared_error(np.log1p(train['Calories']), np.log1p(oof_preds)))
print(f"\nOOF RMSLE: {rmsle:.5f}")

# Submission
sample_submission = pd.read_csv('sample_submission.csv')
submission_xgb = pd.DataFrame({'id': sample_submission.id, 'Calories': y_pred})
submission_xgb.to_csv('submission_xgb_category.csv', index=False)


Fold 1
[0]	validation_0-rmse:0.93366
[100]	validation_0-rmse:0.07504
[200]	validation_0-rmse:0.05973
[300]	validation_0-rmse:0.05971
[329]	validation_0-rmse:0.05970

Fold 2
[0]	validation_0-rmse:0.93360
[100]	validation_0-rmse:0.07596
[200]	validation_0-rmse:0.06053
[300]	validation_0-rmse:0.06047
[354]	validation_0-rmse:0.06047

Fold 3
[0]	validation_0-rmse:0.93571
[100]	validation_0-rmse:0.07493
[200]	validation_0-rmse:0.05897
[300]	validation_0-rmse:0.05892
[359]	validation_0-rmse:0.05897

Fold 4
[0]	validation_0-rmse:0.93432
[100]	validation_0-rmse:0.07456
[200]	validation_0-rmse:0.05903
[300]	validation_0-rmse:0.05902
[358]	validation_0-rmse:0.05905

Fold 5
[0]	validation_0-rmse:0.93583
[100]	validation_0-rmse:0.07580
[200]	validation_0-rmse:0.06047
[300]	validation_0-rmse:0.06043
[348]	validation_0-rmse:0.06043

Fold 6
[0]	validation_0-rmse:0.93168
[100]	validation_0-rmse:0.07577
[200]	validation_0-rmse:0.06021
[300]	validation_0-rmse:0.06017
[330]	validation_0-rmse:0.06023

Fol

In [104]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Setup
X_train = X
y_train = y
X_test = test.drop(columns=['id'])
categorical_cols = ['Sex','Age','Duration']

# Stratify on Duration bins
duration_bins = pd.qcut(train['Duration'], q=10, labels=False)

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Initialize prediction arrays
test_preds_log = np.zeros(len(test))       # accumulate raw predictions (log)
test_preds_lin = np.zeros(len(test))       # accumulate converted predictions
oof_preds = np.zeros(len(train))           # OOF predictions for validation

# Cross-validation loop
for fold, (idx_tr, idx_val) in enumerate(cv.split(X_train, duration_bins)):
    print(f'\nFold {fold + 1}')
    
    X_tr, X_val = X_train.iloc[idx_tr], X_train.iloc[idx_val]
    y_tr, y_val = np.log1p(y_train.iloc[idx_tr]), np.log1p(y_train.iloc[idx_val])
    
    train_pool = Pool(X_tr, y_tr, cat_features=categorical_cols)
    val_pool = Pool(X_val, y_val, cat_features=categorical_cols)
    test_pool = Pool(X_test, cat_features=categorical_cols)


    model = CatBoostRegressor(
        iterations=2000,
        learning_rate=0.03,
        depth=10,
        random_strength=5,
        l2_leaf_reg=7,
        bootstrap_type='Bernoulli',
        grow_policy='SymmetricTree',
        eval_metric='RMSE',
        random_seed=42,
        early_stopping_rounds=50,
        verbose=100
    )
    
    model.fit(train_pool, eval_set=val_pool)
    
    # Predict on test
    preds_test_log = model.predict(test_pool)
    preds_test_lin = np.expm1(preds_test_log)

    test_preds_log += preds_test_log
    test_preds_lin += preds_test_lin

    # OOF predictions for this fold
    oof_preds[idx_val] = np.expm1(model.predict(val_pool))

# Average predictions
test_preds_log = np.expm1(test_preds_log / cv.get_n_splits())  # average in log space
test_preds_lin = test_preds_lin / cv.get_n_splits()            # average in linear space

# Compute OOF RMSLE
rmsle_oof = np.sqrt(mean_squared_error(np.log1p(y_train), np.log1p(oof_preds)))
print(f"\nOOF RMSLE: {rmsle_oof:.5f}")

# Save predictions
test['Calories'] = test_preds_lin
test[['id', 'Calories']].to_csv('catboost_linear_avg.csv', index=False)

test['Calories'] = test_preds_log
test[['id', 'Calories']].to_csv('catboost_log_avg.csv', index=False)


Fold 1


CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=4]=26.0 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [83]:
# CAtboost woth the addiitonal features

['Sex', 'Duration', 'Age']

In [102]:
# Save predictions
test['Calories'] = test_preds_lin
test[['id', 'Calories']].to_csv('catboost_linear_avg_age_cat.csv', index=False)

test['Calories'] = test_preds_log
test[['id', 'Calories']].to_csv('catboost_log_avg_age_cat.csv', index=False)

In [103]:
X_test[categorical_cols].value_counts()

Sex     Age
male    21     6309
female  21     5875
male    32     4606
female  20     4427
        32     4215
               ... 
        77      305
male    77      266
        78      236
        76      199
female  76      165
Name: count, Length: 120, dtype: int64

In [97]:
test.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,bmi,dur_body_heart,...,Age_x_Weight,Age_x_Height,Duration_x_Heart_Rate,Duration_x_Weight,Duration_x_Height,Heart_Rate_x_Weight,Heart_Rate_x_Height,Weight_x_Height,Duration_2,Age_2
0,750000,male,45,177.0,81.0,7.0,87.0,39.8,25.854639,24238.2,...,3645.0,7965.0,609.0,567.0,1239.0,7047.0,15399.0,14337.0,7.0,45
1,750001,male,26,200.0,97.0,20.0,101.0,40.5,24.250000,81810.0,...,2522.0,5200.0,2020.0,1940.0,4000.0,9797.0,20200.0,19400.0,20.0,26
2,750002,female,29,188.0,85.0,16.0,102.0,40.4,24.049344,65932.8,...,2465.0,5452.0,1632.0,1360.0,3008.0,8670.0,19176.0,15980.0,16.0,29
3,750003,female,39,172.0,73.0,20.0,107.0,40.6,24.675500,86884.0,...,2847.0,6708.0,2140.0,1460.0,3440.0,7811.0,18404.0,12556.0,20.0,39
4,750004,female,30,173.0,67.0,16.0,94.0,40.5,22.386314,60912.0,...,2010.0,5190.0,1504.0,1072.0,2768.0,6298.0,16262.0,11591.0,16.0,30
